In [1]:
from dataclasses import dataclass
import numpy as np

# Инициализация количества бит и индентификаторов

In [2]:
M_intM = 3
node_positions: list = [0, 1, 3]
node_positions

[0, 1, 3]

# Классы

In [3]:
@dataclass
class Finger:
    """
    @dataclass - применяется для автоматического добавления
    __init__() и __repr__()
    """
    start: int
    interval: tuple[int, int]
    node_id: int

        
def id_belongs_to_interval(id, a, b, a_included=False, b_included=False):
    """
    Проверка нахождения идентификатора в заданном интервале
    """
    print(f'Проверка {id} на [{a}, {b}]')

    id_less_than_b = id <= b if b_included else id < b
    id_more_than_a = id >= a if a_included else id > a

    if a < b:
        return id_more_than_a and id_less_than_b
    elif a > b:
        return id_more_than_a or id_less_than_b
    else:
        return a_included or b_included


depth = 0

def log(func):
    """
    Вызов функции decorated_func при вызове функции,
    помеченной аннотацией @log
    """
    def decorated_func(self, *args, **kwargs):
        """
        Примитивный способ логирования выполнения функции
        """
        global depth
        tab = lambda msg: '**' * depth + msg
        print(tab('*' * 10))
        print(tab(f'Начинается выполнение {func.__name__}.'))
        print(tab(f'Выполняющая нода №{self.id}.'))
        print(tab(f'Аргументы выполняемой функции: args={args}, kwargs={kwargs}.'))
        depth += 1
        result = func(self, *args, **kwargs)
        depth -= 1
        print(tab(f'Завершение выполнения {func.__name__}.'))
        print(tab(f'Возвращаемое значение: {result}.'))
        print(tab('#' * 10))
        return result
    return decorated_func


class ChordNode:
    def __init__(self, id, finger_table, chord_service, predecessor_id):
        self.id = id
        self.chord_service = chord_service
        self.chord_service.add_node(self)
        
        if finger_table is not None:
            self.finger_table = finger_table
        else: 
            self.finger_table = []
            m = self.chord_service.m
            for i in range(m):
                finger = Finger(None, None, None)
                finger.start = (self.id + 2**i) % 2 ** m
                interval_end = (self.id + 2**(i+1)) % 2 ** m
                finger.interval = (finger.start, interval_end)
                self.finger_table.append(finger)

        self.predecessor_id = predecessor_id
    
    """
    @property - позволяет вызывать метод без передачи аргументов
    """
    @property
    @log
    def successor(self):
        """
        Получение преемника текущей ноды (т.е. первый палец в таблице)
        """
        successor_id = self.finger_table[0].node_id
        return self.chord_service.get_node(successor_id)
    
    @property
    @log
    def predecessor(self):
        """
        Получение предшественника текущей ноды
        """
        return self.chord_service.get_node(self.predecessor_id)
    
    @property
    @log
    def successor_id(self):
        """
        Получение id преемника
        """
        return self.finger_table[0].node_id
    
    @log
    def find_successor(self, id):
        """
        Поиск преемника переданного id.

        Другими словами: поиск преемника предшественника данного id
        """
        if self.id == id:
            return self
        
        node = self.find_predecessor(id)
        return node.successor
    
    @log
    def find_predecessor(self, id):
        """
        Поиск предшественника с помощью ближайших к ней ноде.
        """
        if self.id == id:
            return self.predecessor
        
        node = self
        while not id_belongs_to_interval(id, node.id, node.successor_id, b_included=True):
            node = node.closest_preceding_finger(id)
        return node
    
    @log
    def closest_preceding_finger(self, id):
        """
        Поиск ближайшего пальца, предшествующего id.
        """
        for i in range(len(self.finger_table) - 1, -1, -1):
            finger_id = self.finger_table[i].node_id
            if id_belongs_to_interval(finger_id, self.id, id):
                node = self.chord_service.get_node(finger_id)
                if node is None:
                    self.finger_table[i].node_id = self.id
                    continue
                return self.chord_service.get_node(finger_id)
        return self

    
    @log
    def join(self, other_node):
        """
        Добавление узла в сеть с помощью другой ноды.

        Если другой ноды не существует (добавляемая нода единственная в сети),
        то данная нода добавляется вручную.
        """
        if other_node is not None:
            # Использование другой ноды для добавления текущей
            self.init_finger_table(other_node)
            self.update_others()
        else:
            # Текущая нода единственная
            for i in range(self.chord_service.m):
                self.finger_table[i].node_id = self.id
            self.predecessor_id = self.id
    
    @log
    def init_finger_table(self, other_node):
        """
        Алгоритм анициализации таблицы пальцев добавляемой ноды
        """
        finger_start = self.finger_table[0].start
        succ = other_node.find_successor(finger_start)
        self.finger_table[0].node_id = succ.id
        self.predecessor_id = succ.predecessor_id
        succ.predecessor_id = self.id
        
        for i in range(self.chord_service.m - 1):
            pre_finger = self.finger_table[i]
            finger = self.finger_table[i + 1]
            if id_belongs_to_interval(finger.start, self.id, pre_finger.node_id, a_included=True):
                finger.node_id = pre_finger.node_id
            else:
                finger.node_id = other_node.find_successor(finger.start).id

    @log            
    def update_others(self, removed_id=None):
        """
        Обновление таблиц пальцев у других нод
        """
        m = self.chord_service.m
        for i in range(m):
            pred_id = (self.id - 2**i + 1) % 2**m
            p = self.find_predecessor(pred_id)
            if removed_id is not None and p.id == removed_id:
                p = self.find_predecessor(removed_id)

            p.update_finger_table(self.id, i)
            
    @log
    def disconnect(self):
        """
        Удаление ноды из сети
        """
        self.chord_service.remove_node(self.id)
        succ = self.successor
        pred = self.predecessor
        succ.predecessor_id = pred.id
        pred.finger_table[0].node_id = succ.id
        
        self.successor.update_others(self.id)

    @log
    def update_finger_table(self, s, i): 
        """
        Обновление конкретного пальца у ноды
        """
        if s == self.id:
            return
        
        finger = self.finger_table[i]
        not_accessible = self.chord_service.get_node(finger.node_id) is None

        if id_belongs_to_interval(s, self.id, finger.node_id, a_included=True) or not_accessible:
            finger.node_id = s
            print(f'Обновление пальца {i} в ноде {self.id} при помощи {s}')
            p = self.predecessor_id
            p_node = self.chord_service.get_node(p)
            p_node.update_finger_table(s, i)
        
        if finger.node_id == s :
            print('Обновление пальца {i} в ноде {self.id} не требуется')
            p = self.predecessor_id
            p_node = self.chord_service.get_node(p)
            p_node.update_finger_table(s, i)
        
    def __repr__(self):
        return f'Node: id={self.id}'

In [4]:
class ChordService:
    """
    Данный класс применяется для взаимодействия с окружностью хорд
    """
    def __init__(self, m: int):
        self.n_bits = m
        self.chord_circle = {}
    
    def add_node(self, node: ChordNode):
        self.chord_circle[node.id] = node
    
    def get_node(self, id) -> ChordNode:
        return self.chord_circle.get(id)
    
    def remove_node(self, id):
        self.chord_circle.pop(id)
    
    def init_chord_circle(self, circle):
        for node in circle:
            self.chord_circle[node.id] = node
    
    @property
    def m(self):
        return self.n_bits

# Проверка работоспособности

### Добавление узлов

In [5]:
service = ChordService(M_intM)

In [6]:
node0 = ChordNode(
    id=node_positions[0],
    finger_table=None,
    chord_service=service,
    predecessor_id=None
)

In [7]:
node0.join(None)

**********
Начинается выполнение join.
Выполняющая нода №0.
Аргументы выполняемой функции: args=(None,), kwargs={}.
Завершение выполнения join.
Возвращаемое значение: None.
##########


In [8]:
node1 = ChordNode(
    id=node_positions[1],
    finger_table=None,
    chord_service=service,
    predecessor_id=None
)

In [9]:
node1.join(node0)

**********
Начинается выполнение join.
Выполняющая нода №1.
Аргументы выполняемой функции: args=(Node: id=0,), kwargs={}.
************
**Начинается выполнение init_finger_table.
**Выполняющая нода №1.
**Аргументы выполняемой функции: args=(Node: id=0,), kwargs={}.
**************
****Начинается выполнение find_successor.
****Выполняющая нода №0.
****Аргументы выполняемой функции: args=(2,), kwargs={}.
****************
******Начинается выполнение find_predecessor.
******Выполняющая нода №0.
******Аргументы выполняемой функции: args=(2,), kwargs={}.
******************
********Начинается выполнение successor_id.
********Выполняющая нода №0.
********Аргументы выполняемой функции: args=(), kwargs={}.
********Завершение выполнения successor_id.
********Возвращаемое значение: 0.
********##########
Проверка 2 на [0, 0]
******Завершение выполнения find_predecessor.
******Возвращаемое значение: Node: id=0.
******##########
****************
******Начинается выполнение successor.
******Выполняющая 

In [10]:
node1.finger_table

[Finger(start=2, interval=(2, 3), node_id=0),
 Finger(start=3, interval=(3, 5), node_id=0),
 Finger(start=5, interval=(5, 1), node_id=0)]

In [11]:
node0.finger_table

[Finger(start=1, interval=(1, 2), node_id=1),
 Finger(start=2, interval=(2, 4), node_id=0),
 Finger(start=4, interval=(4, 0), node_id=0)]

In [12]:
node3 = ChordNode(
    id=node_positions[2],
    finger_table=None,
    chord_service=service,
    predecessor_id=None
)

In [13]:
node3.join(node0)

**********
Начинается выполнение join.
Выполняющая нода №3.
Аргументы выполняемой функции: args=(Node: id=0,), kwargs={}.
************
**Начинается выполнение init_finger_table.
**Выполняющая нода №3.
**Аргументы выполняемой функции: args=(Node: id=0,), kwargs={}.
**************
****Начинается выполнение find_successor.
****Выполняющая нода №0.
****Аргументы выполняемой функции: args=(4,), kwargs={}.
****************
******Начинается выполнение find_predecessor.
******Выполняющая нода №0.
******Аргументы выполняемой функции: args=(4,), kwargs={}.
******************
********Начинается выполнение successor_id.
********Выполняющая нода №0.
********Аргументы выполняемой функции: args=(), kwargs={}.
********Завершение выполнения successor_id.
********Возвращаемое значение: 1.
********##########
Проверка 4 на [0, 1]
******************
********Начинается выполнение closest_preceding_finger.
********Выполняющая нода №0.
********Аргументы выполняемой функции: args=(4,), kwargs={}.
Проверка 0 на

In [14]:
node3.finger_table

[Finger(start=4, interval=(4, 5), node_id=0),
 Finger(start=5, interval=(5, 7), node_id=0),
 Finger(start=7, interval=(7, 3), node_id=0)]

In [15]:
node4 = ChordNode(
    id=4,
    finger_table=None,
    chord_service=service,
    predecessor_id=None
)

In [16]:
node4.join(node3)

**********
Начинается выполнение join.
Выполняющая нода №4.
Аргументы выполняемой функции: args=(Node: id=3,), kwargs={}.
************
**Начинается выполнение init_finger_table.
**Выполняющая нода №4.
**Аргументы выполняемой функции: args=(Node: id=3,), kwargs={}.
**************
****Начинается выполнение find_successor.
****Выполняющая нода №3.
****Аргументы выполняемой функции: args=(5,), kwargs={}.
****************
******Начинается выполнение find_predecessor.
******Выполняющая нода №3.
******Аргументы выполняемой функции: args=(5,), kwargs={}.
******************
********Начинается выполнение successor_id.
********Выполняющая нода №3.
********Аргументы выполняемой функции: args=(), kwargs={}.
********Завершение выполнения successor_id.
********Возвращаемое значение: 0.
********##########
Проверка 5 на [3, 0]
******Завершение выполнения find_predecessor.
******Возвращаемое значение: Node: id=3.
******##########
****************
******Начинается выполнение successor.
******Выполняющая 

In [17]:
node4.finger_table

[Finger(start=5, interval=(5, 6), node_id=0),
 Finger(start=6, interval=(6, 0), node_id=0),
 Finger(start=0, interval=(0, 4), node_id=0)]

In [18]:
node3.finger_table

[Finger(start=4, interval=(4, 5), node_id=4),
 Finger(start=5, interval=(5, 7), node_id=0),
 Finger(start=7, interval=(7, 3), node_id=0)]

In [19]:
node1.finger_table

[Finger(start=2, interval=(2, 3), node_id=3),
 Finger(start=3, interval=(3, 5), node_id=3),
 Finger(start=5, interval=(5, 1), node_id=0)]

In [20]:
node0.finger_table

[Finger(start=1, interval=(1, 2), node_id=1),
 Finger(start=2, interval=(2, 4), node_id=3),
 Finger(start=4, interval=(4, 0), node_id=4)]

### Удаление ноды

In [21]:
node3.disconnect()

**********
Начинается выполнение disconnect.
Выполняющая нода №3.
Аргументы выполняемой функции: args=(), kwargs={}.
************
**Начинается выполнение successor.
**Выполняющая нода №3.
**Аргументы выполняемой функции: args=(), kwargs={}.
**Завершение выполнения successor.
**Возвращаемое значение: Node: id=4.
**##########
************
**Начинается выполнение predecessor.
**Выполняющая нода №3.
**Аргументы выполняемой функции: args=(), kwargs={}.
**Завершение выполнения predecessor.
**Возвращаемое значение: Node: id=1.
**##########
************
**Начинается выполнение successor.
**Выполняющая нода №3.
**Аргументы выполняемой функции: args=(), kwargs={}.
**Завершение выполнения successor.
**Возвращаемое значение: Node: id=4.
**##########
************
**Начинается выполнение update_others.
**Выполняющая нода №4.
**Аргументы выполняемой функции: args=(3,), kwargs={}.
**************
****Начинается выполнение find_predecessor.
****Выполняющая нода №4.
****Аргументы выполняемой функции: arg

In [22]:
node4.finger_table

[Finger(start=5, interval=(5, 6), node_id=0),
 Finger(start=6, interval=(6, 0), node_id=0),
 Finger(start=0, interval=(0, 4), node_id=0)]

In [23]:
node3.finger_table

[Finger(start=4, interval=(4, 5), node_id=4),
 Finger(start=5, interval=(5, 7), node_id=0),
 Finger(start=7, interval=(7, 3), node_id=0)]

In [24]:
node1.finger_table

[Finger(start=2, interval=(2, 3), node_id=4),
 Finger(start=3, interval=(3, 5), node_id=4),
 Finger(start=5, interval=(5, 1), node_id=0)]

In [25]:
node0.finger_table

[Finger(start=1, interval=(1, 2), node_id=1),
 Finger(start=2, interval=(2, 4), node_id=4),
 Finger(start=4, interval=(4, 0), node_id=4)]